## Exporting to TensorFlow
You can use the Fireball's [exportToTf](https://interdigitalinc.github.io/Fireball/html/source/model.html#fireball.model.Model.exportToTf) function to export a model to TensorFlow code. This function creates a
python file that implements the model using TensorFlow APIs. It also creates a numpy file (npz) that contains the parameters of the network. This notebook shows how to use this function to export a Fireball model to TensorFlow. It assumes that a trained LeNet-5 model already exists in the ```Models``` directory. You can use the notebook [Handwritten Digit recognition (LeNet-5/MNIST)](LeNet5-MNIST.ipynb) to create and train a LeNet-5 model.

Fireball can also export models with reduced number of parameters, pruned models, and quatized models. Please refer to the following notebooks for more information:

- [Reducing number of parameters of LeNet-5 Model](LeNet5-MNIST-Reduce.ipynb)
- [Pruning LeNet-5 Model](LeNet5-MNIST-Prune.ipynb)
- [Quantizing LeNet-5 Model](LeNet5-MNIST-Quantize.ipynb)


## Load a pretrained model

In [1]:
from fireball import Model
from fireball.datasets.mnist import MnistDSet

testDs = MnistDSet.makeDatasets('test', batchSize=128)

# orgFileName = "Models/LeNet5RRPRQR.fbm"  # Reduced - Retrained - Pruned - Retrained - Quantized - Retrained
orgFileName = "Models/LeNet5RRPR.fbm"  # Reduced - Retrained - Pruned - Retrained  (Use this when training in TF)

model = Model.makeFromFile(orgFileName, testDs=testDs, gpus='0')   
model.initSession()
model.printLayersInfo()

results = model.evaluate()


Reading from "Models/LeNet5RRPR.fbm" ... Done.
Creating the fireball model "LeNet-5" ... Done.

Scope            InShape       Comments                 OutShape      Activ.   Post Act.        # of Params
---------------  ------------  -----------------------  ------------  -------  ---------------  -----------
IN_IMG                         Image Size: 28x28x1      28 28 1       None                      0          
L1_CONV          28 28 1       KSP: 5 1 s               14 14 6       ReLU     MP(KSP):2 2 v    108        
L2_CONV          14 14 6       KSP: 5 1 v, LR8          5 5 16        ReLU     MP(KSP):2 2 v    836        
L3_FC            5 5 16        LR8                      120           ReLU                      2,390      
L4_FC            120           LR8                      84            ReLU                      959        
L5_FC            84                                     10            None                      525        
OUT_CLASS        10            10 class

## Export the model
Fireball creates a folder and puts 2 files in the folder. Here we call the [exportToTf](https://interdigitalinc.github.io/Fireball/html/source/model.html#fireball.model.Model.exportToTf) funtion to export the model.

In [2]:

model.exportToTf("Models/LeNet5TF", runQuantized=True, classNames=[str(i) for i in range(10)])



Exporting to TensorFlow model "Models/LeNet5TF" ... 
    Processed all 7 layers.                                     
    Creating parameters file "Params.npz" ... Done.
Done.


Now we have the exported model in the folder ``Models/LeNet5TF``. Inside this folder there is a python file "TfModel.py" that was created by Fireball. Open this file and review the code generated by Fireball. This file defines a class called ```Network``` which implements the exported Fireball model.

## Evaluating the exported model
We can now evaluate the exported model. Before running the next cell, reset the kernel to make sure there is no dependency to the Fireball library.

In [3]:
import numpy as np
import struct

def getDataset(imagesFileName, labelsFileName):
    file = open(imagesFileName, mode='rb')
    header = file.read(16)
    magic, numImages, imageWidth, imageHeight = struct.unpack(">iiii", header)
    assert magic == 2051, "Error: Invalid MNIST Image format!"

    buf = file.read(imageWidth * imageHeight * numImages)
    data = np.frombuffer(buf, dtype=np.uint8).astype(np.float32)
    data = (data-127.5)/255.0   # Normalize to [-1..1]
    samples = data.reshape(numImages, imageWidth, imageHeight, 1)

    file = open(labelsFileName, mode='rb')
    header = file.read(8)
    magic, numLabels = struct.unpack(">ii", header)
    assert magic == 2049, "Error: Invalid MNIST Label format!"

    buf = file.read(numLabels)
    labels = np.frombuffer(buf, dtype=np.uint8).astype(np.int64)
    return samples, labels

# Update the file names to point to the location of MNIST dataset
testSamples, testLabels = getDataset('/data/mnist/t10k-images.idx3-ubyte',
                                     '/data/mnist/t10k-labels.idx1-ubyte')

# Now we import the Network class that was generated by Fireball in the "TfModel.py" file
from Models.LeNet5TF.TfModel import Network
net=Network()

The following cell shows how to run inference. We get a random test sample and call the ```infer``` function to get the probabilities of different classes in an array. The ```argmax``` function gives us the actual label. You can run it several time to test it with different samples.

In [4]:
i = np.random.randint(len(testLabels))
print( "Actual label of the sample no %d in the dataset: %d"%(i, testLabels[i]))
classProbs = net.infer(testSamples[i:i+1])
print( "Predicted label: %d"%(np.argmax(classProbs)))

Actual label of the sample no 1156 in the dataset: 7
Predicted label: 7


Now we can run the evaluation over all test samples.

In [5]:
classProbs = net.infer(testSamples)
accuracy = float(np.sum(testLabels == np.argmax(classProbs,1)))/len(testLabels)
print( "Test Accuracy: %f"%(accuracy) )

Test Accuracy: 0.991600


## Training the exported model in TensorFlow
The exported model also includes everything that is needed for training a model. You just need to create your own optimizer (using TensorFlow) and pass it to the ```trainBatch``` function of the ```Network``` class. The following cell shows an example of how to train an exported model.

Please note that when the ```Network``` class is instantiated for training, the exported parameter values are not used. The parameters of the network are initialized randomly.

**IMPORTANT:** Do not use a quantized model for training. Quantization must happen after training. So, please scroll up and make sure the quantized model file (Models/LeNet5RRQR.fbm) is not selected in the first cell of this notebook.

In [6]:
import tensorflow as tf

# NOTE: Make sure to update this with the same file used above in the first cell
orgFileName = "Models/LeNet5RRPR.fbm"    # Reduced - Retrained - Pruned - Retrained

# This works for Non-Quantized models only!
assert ('QR.fbm' not in orgFileName) and ('Q.fbm' not in orgFileName), "No Training for Quanized Models!"

# first create the training dataset:
trainSamples, trainLabels = getDataset('/data/mnist/train-images.idx3-ubyte',
                                       '/data/mnist/train-labels.idx1-ubyte')

# Create an instance of the Network for training:
net = Network(train=True)

# Create a learning rate and a gradient descent optimizer:
startLearningRate = 0.1
learningRate = tf.compat.v1.train.exponential_decay(startLearningRate,
                                                    net.globalStep,  # Use the "globalStep" from the exported model
                                                    50,
                                                    0.96,
                                                    staircase=True)
optimizer = tf.compat.v1.train.GradientDescentOptimizer(learningRate)
optimize = optimizer.minimize(net.loss, global_step=net.globalStep)

# Train for 10 epochs:
sampleIndexes = np.arange(len(trainLabels))
numEpochs = 10
batchSize = 128

for e in range(numEpochs):
    np.random.shuffle(sampleIndexes)
    numBatches = len(trainLabels)//batchSize
    for b in range(numBatches):
        batchSamples = trainSamples[sampleIndexes[b*batchSize:(b+1)*batchSize]]
        batchLabels = trainLabels[sampleIndexes[b*batchSize:(b+1)*batchSize]]
        
        net.trainBatch(optimize, batchSamples, batchLabels)
    
    lr = net.session.run(learningRate)
    classProbs = net.infer(testSamples)
    accuracy = float(np.sum(testLabels == np.argmax(classProbs,1)))/len(testLabels)
    print("Epoch %d -> Learning Rate: %f, Test Accuracy: %f"%(e+1, lr, accuracy))

Epoch 1 -> Learning Rate: 0.069253, Test Accuracy: 0.955600
Epoch 2 -> Learning Rate: 0.047960, Test Accuracy: 0.971000
Epoch 3 -> Learning Rate: 0.031886, Test Accuracy: 0.979300
Epoch 4 -> Learning Rate: 0.022082, Test Accuracy: 0.978900
Epoch 5 -> Learning Rate: 0.015292, Test Accuracy: 0.979800
Epoch 6 -> Learning Rate: 0.010167, Test Accuracy: 0.981100
Epoch 7 -> Learning Rate: 0.007041, Test Accuracy: 0.980700
Epoch 8 -> Learning Rate: 0.004876, Test Accuracy: 0.980700
Epoch 9 -> Learning Rate: 0.003242, Test Accuracy: 0.981300
Epoch 10 -> Learning Rate: 0.002245, Test Accuracy: 0.981300


## Also look at

[Exporting LeNet-5 Model to ONNX](LeNet5-MNIST-ONNX.ipynb)

[Exporting LeNet-5 Model to CoreML](LeNet5-MNIST-CoreML.ipynb)

---

[Fireball Playgrounds](../Contents.ipynb)

[Handwritten Digit Recognition (LeNet-5/MNIST)](LeNet5-MNIST.ipynb)

[Reducing number of parameters of LeNet-5 Model](LeNet5-MNIST-Reduce.ipynb)

[Pruning LeNet-5 Model](LeNet5-MNIST-Prune.ipynb)

[Quantizing LeNet-5 Model](LeNet5-MNIST-Quantize.ipynb)